In [53]:
import requests
import json
import pandas as pd

In [54]:
sky_url = "https://election.news.sky.com/api/elections/get-data?queryKey=ELECTIONS_DATA_QUERY&electionId=2024-07-04-UK_GENERAL&electionType=UK_GENERAL&"
req = requests.get(sky_url)
data = json.loads(req.text)
data = data['election']


In [55]:
df = pd.DataFrame(data['votingAreas'])
df = df.query("votingStatus != 'BASELINE'")
df['winner'] = df.electedCandidate.apply(lambda x: x['party']['name'])
results_df = df.copy()

results_df['previous'] = df.previouslyElectedCandidate.apply(lambda x: x['party']['name'] if x else None)

## For the Sankey

In [69]:
df = results_df[['winner', 'previous', 'id']].groupby(['previous', 'winner']).agg('count').reset_index().rename(columns={'id': 'count'}).to_csv('from_to_results.csv', index=False)
df = results_df[['winner', 'previous', 'id']].groupby(['previous', 'winner']).agg('count').reset_index().rename(columns={'id': 'value'})
df['winner'] = df['winner'] + ' '

# fill through each previous-winner pair with 0 if it doesn't exist
for prev in results_df['previous'].unique():
    for winner in results_df['winner'].unique():
        if not ((df['previous'] == prev) & (df['winner'] == winner)).any():
            df = pd.concat([df, pd.DataFrame([[prev, winner, 0]], columns=['previous', 'winner', 'value'])])

df = df.rename(columns={'previous': 'source', 'winner': 'target'})
df.to_csv('/Users/finn/Downloads/from_to_results.csv', index=False)

In [71]:
df

,source,target,value
0,Conservative,Conservative,3
1,Conservative,Labour,16
2,Conservative,Liberal Democrat,2
3,Conservative,Reform UK,1
4,Labour,Labour,24
0,Labour,Labour,0
0,Labour,Conservative,0
0,Labour,Liberal Democrat,0
0,Labour,Reform UK,0
0,Conservative,Labour,0


In [ ]:
results_df.top

In [27]:
df = pd.read_csv("exit_poll_data.csv")
exit_df = df.copy()

In [31]:
exit_df[exit_df.name.str.contains('Washington')]

,name,forecasts,previousWinner,change,code
127,Washington and Gateshead South,Labour,NaN,True,E14001567


In [33]:
exit_df

,name,forecasts,previousWinner,change,code
0,Guildford,Liberal Democrat,NaN,True,E14001258
1,Mitcham and Morden,Labour,NaN,True,E14001371
2,Newcastle upon Tyne East and Wallsend,Labour,NaN,True,E14001378
3,Scunthorpe,Labour,NaN,True,E14001462
4,Norwich North,Labour,NaN,True,E14001408
...,...,...,...,...,...
645,South Down,NaN,NaN,NaN,N05000015
646,Belfast North,NaN,NaN,NaN,N05000002
647,Belfast East,NaN,NaN,NaN,N05000001
648,Strangford,NaN,NaN,NaN,N05000016


In [36]:
exit_df['name'] = exit_df['name'].str.replace(' & ', ' and ')
results_df['name'] = results_df['name'].str.replace(' & ', ' and ')
df = pd.merge(results_df, exit_df, on='name', indicator=True, how='left')
df = df[['name', 'shortName', 'winner', 'forecasts', 'gainOrHold']]
df

,name,shortName,winner,forecasts,gainOrHold
0,Sunderland Central,Sunderland Central,Labour,Labour,HOLD
1,Telford,Telford,Labour,Labour,GAIN
2,Nuneaton,Nuneaton,Labour,Labour,GAIN
3,Washington and Gateshead South,Washington & Gateshead South,Labour,Labour,HOLD
4,Swindon South,Swindon South,Labour,Labour,GAIN
5,Rayleigh and Wickford,Rayleigh & Wickford,Conservative,Too close to call,HOLD
6,Knowsley,Knowsley,Labour,Labour,HOLD
7,Swindon North,Swindon North,Labour,Labour,GAIN
8,Eastleigh,Eastleigh,Liberal Democrat,Liberal Democrat,GAIN
9,South Shields,South Shields,Labour,Labour,HOLD
